In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.externals import joblib

In [2]:
from metricas import smape

In [3]:
from preprocessamento import *

In [5]:
serie_name = 'star'
print('Série:', serie_name)
endereco = serie_name+'.txt'
dados = pd.read_csv(endereco, delimiter=' ', header=None)
serie = dados[0]
serie_normalizada = normalise(serie)

train, test = split_serie_less_lags(serie_normalizada, 0.75)

max_lag = 20
lags_acf = select_lag_acf(serie_normalizada, max_lag)
max_sel_lag = lags_acf[0]

train_lags = create_windows(train, max_sel_lag+1)

test_data = create_windows(test, max_sel_lag+1)

train_data,  val_data = select_validation_sample(train_lags, 0.34)

Série: star
LAGS [0, 1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [6]:
len(serie)

600

In [7]:
x_train = train_data[:,0:-1]
x_train = x_train[:,lags_acf]  #retorna só os lags selecionados no acf
y_train = train_data[:,-1]
x_val = val_data[:,0:-1]
x_val = x_val[:,lags_acf]
y_val = val_data[:,-1]
x_test = test_data[:,lags_acf]
y_test = test_data[:,-1]

In [ ]:
def gerar_lstm(neuronios,  lags, func_opt='adam'):
    from keras.models import Sequential
    from keras.layers import LSTM, Dense, Dropout

    model = Sequential()
    model.add(LSTM(neuronios, input_shape=(1, lags)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=func_opt)

    return model

In [ ]:
def train_lstm(modelo, x_train, y_train, x_val, y_val, num_ex=5,epochs=100):
    trainX = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
    valX = np.reshape(x_val, (x_val.shape[0], 1, x_val.shape[1]))
    melhor_mse = np.Inf
    for i in range(0, num_ex):

        modelo.fit(trainX, y_train, epochs=epochs, batch_size=1, verbose=0)
        prev_v = modelo.predict(valX)
        novo_mse  = MSE(y_val, prev_v)
        if novo_mse < melhor_mse:
            melhor_mse = novo_mse
            melhor_modelo = modelo

    return melhor_modelo, melhor_mse

In [ ]:
def prev_lstm(modelo, x_test):
    testX = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
    y_prev = modelo.predict(testX)
    previsao = []
    for i in range(0, len(y_prev)):
        previsao.append(y_prev[i][0])


    return previsao

In [8]:
neuronios = [1, 10, 20, 50,  100]
func_opt = ['adam','sgd', 'rmsprop']
best_result = np.Inf

for i in range(0,len(neuronios)):
    for j in range(0,len(func_opt)):
        lstm = gerar_lstm(neuronios[i], len(lags_acf), func_opt = func_opt[j])
        lstm_treinada, mse_val = train_lstm(lstm, x_train, y_train, x_val, y_val, num_ex=1,epochs=20)
        if mse_val < best_result:
            best_result = mse_val
            select_model = lstm_treinada
            print('melhor configuração. neuronios:', neuronios[i], 'funcao:', func_opt[j])
        


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


melhor configuração. neuronios: 1 funcao: adam
melhor configuração. neuronios: 1 funcao: sgd


In [9]:
filename = serie_name+'_LSTM.h5'
select_model.save(filename)
